In [1]:
import os
print(os.getcwd())

work_directory_R = os.getcwd() + "/Results_ROC"
work_directory_C = os.getcwd() + "/Results_CM"
work_directory_P = os.getcwd() + "/ResultsPredicted"
Results_File = os.getcwd() + "/Metrics_Result.txt"

import shutil

## Try to remove tree; if failed show an error using try...except on screen
try:
    shutil.rmtree(work_directory_R)
    shutil.rmtree(work_directory_C)
    shutil.rmtree(work_directory_P)
    os.remove(Results_File)
except OSError as e:
    print ("Error: %s - %s." % (e.filename, e.strerror))
    
import gc as g

# %reset
from IPython import get_ipython
get_ipython().magic('reset -sf') 

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)

import tensorflow as tf,keras
from keras import backend as K, optimizers, regularizers
# K.clear_session()

import pickle, sys,os,cv2, numpy as np, pandas as pd, scipy, seaborn as sns, json, joblib, sklearn as sk
from random import shuffle
from keras.models import Sequential, Model, load_model, model_from_json
# from keras.layers import Dense, Permute, Dropout, Activation, Flatten, Conv2D, MaxPool2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Input, ZeroPadding2D, merge
from keras.layers import Dense, Dropout, Permute, Activation, Flatten, Conv2D, MaxPool2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Input, ZeroPadding2D, merge
from keras.layers import SeparableConv2D, DepthwiseConv2D, BatchNormalization, SpatialDropout2D
from keras.layers.merge import concatenate
from keras import utils as np_utils
from keras.regularizers import l2, l1_l2
from keras.constraints import max_norm

from keras.optimizers import Adam,RMSprop,SGD
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator,array_to_img
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, EarlyStopping

import matplotlib.pyplot as plt
import matplotlib.cm as cm

get_ipython().magic('matplotlib inline')
%matplotlib inline

from keras import layers
# from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.model_selection import StratifiedKFold, GroupKFold, RepeatedKFold, train_test_split, GridSearchCV
from sklearn.metrics import cohen_kappa_score, accuracy_score, confusion_matrix, roc_curve, auc, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler


# from keras.utils.training_utils import multi_gpu_model
# from keras.utils import multi_gpu_model
# from keras import layers
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
# from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
import joblib


from tqdm import tqdm
from functools import partial
from sklearn import metrics
from collections import Counter
from itertools import cycle
from six.moves import cPickle as pickle
from scipy import interp
from skimage.feature import local_binary_pattern,hog
from skimage import data, exposure

import itertools
from time import time
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from radiomics import featureextractor  # This module is used for interaction with pyradiomics
import radiomics
# import SimpleITK as sitk
import pkgutil
import xlsxwriter
from PIL import Image
from natsort import natsorted

print('Libraries Loaded')

print()
print(f"Python Version: {sys.version}")
print(f"Tensor Flow Version: {tf.__version__}")
# print(f"Keras Version: {tf.keras.__version__}")
print(f"Keras Version: {keras.__version__}")
print(f"Numpy Version: {np.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"OpenCV Version: {cv2.__version__}")
print(f"Scikit-Learn Version: {sk.__version__}")

tf.__version__
tf.test.is_built_with_cuda()

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

# GPU Check 1
from tensorflow.python.client import device_lib
device_lib.list_local_devices()


# from keras import backend as K
# print(K.tensorflow_backend._get_available_gpus())

D:\Research Work\Paper - 4\BreastLiverNet\MoNuSAC\My_Code 19-05-2022 Professor Idea\Final Code\Decemeber_22
Error: D:\Research Work\Paper - 4\BreastLiverNet\MoNuSAC\My_Code 19-05-2022 Professor Idea\Final Code\Decemeber_22/Results_ROC - The system cannot find the path specified.


Using TensorFlow backend.


Libraries Loaded

Python Version: 3.6.13 |Anaconda, Inc.| (default, Mar 16 2021, 11:37:27) [MSC v.1916 64 bit (AMD64)]
Tensor Flow Version: 1.15.0
Keras Version: 2.2.4
Numpy Version: 1.19.2
Pandas Version: 1.1.5
OpenCV Version: 4.6.0
Scikit-Learn Version: 0.24.2


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16929344671444853047,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15125499085
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1782287086337258965
 physical_device_desc: "device: 0, name: Quadro RTX 5000, pci bus id: 0000:17:00.0, compute capability: 7.5"]

In [16]:
def Mode(Organ, work_directory_Pickles, option_Mode):

    if option_Mode==1:
        Mode = "Train"
        data_file1 = work_directory_Pickles + '/Classify_C_' + str(Mode) + '_' + str(Organ) + '.pickle'
        data_file2 = ""
        
    elif option_Mode==2:
        Mode = "Test"
        data_file1 = work_directory_Pickles + '/Classify_C_' + str(Mode) + '_' + str(Organ) + '.pickle'
        data_file2 = ""
        
    elif option_Mode==3:
        Mode = "Train"
        data_file1 = work_directory_Pickles + '/Classify_C_' + str(Mode) + '_' + str(Organ) + '.pickle'

        Mode = "Test"
        data_file2 = work_directory_Pickles + '/Classify_C_' + str(Mode) + '_' + str(Organ) + '.pickle'
  
    return data_file1, data_file2 


def load_pickle(data_file,q):
    print('\nTring to load pickle from %s' % data_file)
    with open(data_file, 'rb') as file:
        datasets = pickle.load(file)
        dataset = datasets['dataset']

    X_train = dataset['X_train']
    Y_train = dataset['Y_train']
    
    print('\nPickle Loaded Successfully!')
    
    del dataset

    if q==1:
        print('\nX_train shape:', X_train.shape)
        print('Y_train shape:', Y_train.shape)
    
    return X_train,Y_train


def load_pickle__(data_file,q):
    print('\nTring to load pickle from %s' % data_file)
    with open(data_file, 'rb') as file:
        datasets = pickle.load(file)
        dataset = datasets['dataset']

    X_train = dataset['X_train']
    Y_train = dataset['Y_train']
    Y_train_ = dataset['Y_train_']
    
    print('\nPickle Loaded Successfully!')
    
    del dataset

    if q==1:
        print('\nX_train shape:', X_train.shape)
        print('Y_train shape:', Y_train.shape)
        print('Y_train shape:', Y_train_.shape)
    
    return X_train,Y_train,Y_train_


def save_pickle(x_train,y_train, data_file):
    print('\nTrying to save pickle to %s' % data_file)
    
    X_train = x_train
    Y_train = y_train
    del x_train,y_train
    
    # creating dictionary to store trian and test data
    datasets = {'dataset' : {'X_train': X_train,'Y_train': Y_train}}

    with open(data_file, 'wb') as file:
        pickle.dump(datasets, file, protocol=pickle.HIGHEST_PROTOCOL)
        del datasets # to free up memory.
        
    print('\nPickle Saved Successfully!')
    

def save_pickle__(x_train,y_train,y_train_, data_file):
    print('\nTrying to save pickle to %s' % data_file)
    
    X_train = x_train
    Y_train = y_train
    Y_train_ = y_train_
    del x_train,y_train,y_train_
    
    # creating dictionary to store trian and test data
    datasets = {'dataset' : {'X_train': X_train,'Y_train': Y_train,'Y_train_': Y_train_}}

    with open(data_file, 'wb') as file:
        pickle.dump(datasets, file, protocol=pickle.HIGHEST_PROTOCOL)
        del datasets # to free up memory.
        
    print('\nPickle Saved Successfully!')
    

In [17]:
K.clear_session()
import gc
gc.collect()

49

In [29]:
work_directory_Pickles = os.path.join(os.getcwd() + "/January/PICKLESNEW")

print("\n\n DataSet-MONUSAC \n")

print('\n\nSelect Organ')
option_Org = int(input("\n 1-Breast \n 2-Kidney \n 3-Lung \n 4-Prostate \n 5-All \n"))
while option_Org  not in (1,2,3,4,5):
    option_Org = int(input("\n 1-Breast \n 2-Kidney \n 3-Lung \n 4-Prostate \n 5-All \n"))

    
print('\n\nSelect Train or Test')
option_Mode = int(input("\n 1-Train \n 2-Test \n"))
while option_Mode  not in (1,2):
    option_Mode = int(input("\n 1-Train \n 2-Test \n"))

Organ_B = "Breast"
Organ_K = "Kidney"
Organ_L = "Lung"
Organ_P = "Prostate"


Organ_B_ = "Breast_I"
Organ_K_ = "Kidney_I"
Organ_L_ = "Lung_I"
Organ_P_ = "Prostate_I"


    
if option_Org==1:
    data_file, _ = Mode(Organ_B, work_directory_Pickles, option_Mode)
    Organ_Output_B, Organ_Label_B = load_pickle(data_file,1)

    data_file_, _ = Mode(Organ_B_, work_directory_Pickles, option_Mode)
    Organ_Output_B_, Organ_Label_B_, Organ_Label_B__ = load_pickle__(data_file_,1)


elif option_Org==2:
    data_file, _ = Mode(Organ_K, work_directory_Pickles, option_Mode)
    Organ_Output_K, Organ_Label_K = load_pickle(data_file,1)

    data_file_, _ = Mode(Organ_K_, work_directory_Pickles, option_Mode)
    Organ_Output_K_, Organ_Label_K_, Organ_Label_K__ = load_pickle__(data_file_,1)


elif option_Org==3:
    data_file, _ = Mode(Organ_L, work_directory_Pickles, option_Mode)
    Organ_Output_L, Organ_Label_L = load_pickle(data_file,1)

    data_file_, _ = Mode(Organ_L_, work_directory_Pickles, option_Mode)
    Organ_Output_L_, Organ_Label_L_, Organ_Label_L__ = load_pickle__(data_file_,1)

    
elif option_Org==4:
    data_file, _ = Mode(Organ_P, work_directory_Pickles, option_Mode)
    Organ_Output_P, Organ_Label_P = load_pickle(data_file,1)

    data_file_, _ = Mode(Organ_P_, work_directory_Pickles, option_Mode)
    Organ_Output_P_, Organ_Label_P_, Organ_Label_P__ = load_pickle__(data_file_,1)

    
elif option_Org==5:
    
    data_file, _ = Mode(Organ_B, work_directory_Pickles, option_Mode)
    Organ_Output_B, Organ_Label_B = load_pickle(data_file,1)

    data_file, _ = Mode(Organ_K, work_directory_Pickles, option_Mode)
    Organ_Output_K, Organ_Label_K = load_pickle(data_file,1)

    data_file, _ = Mode(Organ_L, work_directory_Pickles, option_Mode)
    Organ_Output_L, Organ_Label_L = load_pickle(data_file,1)

    data_file, _ = Mode(Organ_P, work_directory_Pickles, option_Mode)
    Organ_Output_P, Organ_Label_P = load_pickle(data_file,1)


    data_file_, _ = Mode(Organ_B_, work_directory_Pickles, option_Mode)
    Organ_Output_B_, Organ_Label_B_, Organ_Label_B__ = load_pickle__(data_file_,1)

    data_file_, _ = Mode(Organ_K_, work_directory_Pickles, option_Mode)
    Organ_Output_K_, Organ_Label_K_, Organ_Label_K__ = load_pickle__(data_file_,1)

    data_file_, _ = Mode(Organ_L_, work_directory_Pickles, option_Mode)
    Organ_Output_L_, Organ_Label_L_, Organ_Label_L__ = load_pickle__(data_file_,1)

    data_file_, _ = Mode(Organ_P_, work_directory_Pickles, option_Mode)
    Organ_Output_P_, Organ_Label_P_, Organ_Label_P__ = load_pickle__(data_file_,1)


    



 DataSet-MONUSAC 



Select Organ

 1-Breast 
 2-Kidney 
 3-Lung 
 4-Prostate 
 5-All 
4


Select Train or Test

 1-Train 
 2-Test 
1

Tring to load pickle from D:\Research Work\Paper - 4\BreastLiverNet\MoNuSAC\My_Code 19-05-2022 Professor Idea\Final Code\Decemeber_22/January/PICKLESNEW/Classify_C_Train_Prostate.pickle

Pickle Loaded Successfully!

X_train shape: (975, 107)
Y_train shape: (975,)

Tring to load pickle from D:\Research Work\Paper - 4\BreastLiverNet\MoNuSAC\My_Code 19-05-2022 Professor Idea\Final Code\Decemeber_22/January/PICKLESNEW/Classify_C_Train_Prostate_I.pickle

Pickle Loaded Successfully!

X_train shape: (975, 128, 128, 3)
Y_train shape: (975, 128, 128)
Y_train shape: (975,)


In [30]:
def Indices_Type(Organ_Output, Organ_Label, Type_Value, ):

    print("\n Labels Portion \n")
    unique, counts = np.unique(Organ_Label, return_counts=True)
    abc= dict(zip(unique, counts))
    print(abc)
    for i in range(len(abc)):
        print('Class %d: %d' % (i, abc[i]))

    result_F = np.where(Organ_Label == Type_Value)
    
    Organ_Output, Organ_Label = Organ_Output[result_F], Organ_Label[result_F]
    
    return Organ_Output, Organ_Label, result_F


print('\n\n\nSelect Class')
option_Class = int(input("\n 1-Epithelial \n 2-Lymphocytes \n 3-Macrophoges \n 4-Neutrophils \n"))
while option_Class  not in (1,2,3,4):
    option_Class = int(input("\n 1-Epithelial \n 2-Lymphocytes \n 3-Macrophoges \n 4-Neutrophils \n"))

if option_Class==1:
    Class = "Epithelial"
    Type_Value = 0
    
elif option_Class==2:
    Class = "Lymphocytes"
    Type_Value = 1
    
elif option_Class==3:
    Class = "Macrophoges"
    Type_Value = 2
    
elif option_Class==4:
    Class = "Neutrophils"
    Type_Value = 3
    
    
if option_Org==1:
    Nuceli_Output_Type_B, Nuceli_Label_Type_B, result_Indices_B = Indices_Type(Organ_Output_B, Organ_Label_B, Type_Value)
    print(Nuceli_Label_Type_B.shape)
    
    Nuceli_Output_Type = Nuceli_Output_Type_B
    Nuceli_Label_Type = Nuceli_Label_Type_B

    samples_Select = 70
    Nuceli_Output_Type = Nuceli_Output_Type[:, 32:102]

    print('\n\nX_train shape:', Nuceli_Output_Type.shape)
    print('Y_train shape:', Nuceli_Label_Type.shape) 

    Nuceli_Output_Type_B_I, Nuceli_Label_Type_B_I, Nuceli_Label_Type_B_I_ = Organ_Output_B_[result_Indices_B], Organ_Label_B_[result_Indices_B], Organ_Label_B__[result_Indices_B]
    print(Nuceli_Label_Type_B_I.shape)

    Nuceli_Output_Type_I = Nuceli_Output_Type_B_I
    Nuceli_Label_Type_I = Nuceli_Label_Type_B_I
    Nuceli_Label_Type_I_ = Nuceli_Label_Type_B_I_
    
    
elif option_Org==2:
    Nuceli_Output_Type_K, Nuceli_Label_Type_K, result_Indices_K = Indices_Type(Organ_Output_K, Organ_Label_K, Type_Value)
    print(Nuceli_Label_Type_K.shape)

    Nuceli_Output_Type = Nuceli_Output_Type_K   
    Nuceli_Label_Type = Nuceli_Label_Type_K
    
    samples_Select = 70
    Nuceli_Output_Type = Nuceli_Output_Type[:, 32:102]

    print('\n\nX_train shape:', Nuceli_Output_Type.shape)
    print('Y_train shape:', Nuceli_Label_Type.shape) 

    Nuceli_Output_Type_K_I, Nuceli_Label_Type_K_I, Nuceli_Label_Type_K_I_ = Organ_Output_K_[result_Indices_K], Organ_Label_K_[result_Indices_K], Organ_Label_K__[result_Indices_K]
    print(Nuceli_Label_Type_K_I.shape)
    
    Nuceli_Output_Type_I = Nuceli_Output_Type_K_I
    Nuceli_Label_Type_I = Nuceli_Label_Type_K_I
    Nuceli_Label_Type_I_ = Nuceli_Label_Type_K_I_
    

elif option_Org==3:
    Nuceli_Output_Type_L, Nuceli_Label_Type_L, result_Indices_L = Indices_Type(Organ_Output_L, Organ_Label_L, Type_Value)
    print(Nuceli_Label_Type_L.shape)

    Nuceli_Output_Type = Nuceli_Output_Type_L
    Nuceli_Label_Type = Nuceli_Label_Type_L

    samples_Select = 70
    Nuceli_Output_Type = Nuceli_Output_Type[:, 32:102]

    print('\n\nX_train shape:', Nuceli_Output_Type.shape)
    print('Y_train shape:', Nuceli_Label_Type.shape) 

    Nuceli_Output_Type_L_I, Nuceli_Label_Type_L_I, Nuceli_Label_Type_L_I_ = Organ_Output_L_[result_Indices_L], Organ_Label_L_[result_Indices_L], Organ_Label_L__[result_Indices_L]
    print(Nuceli_Label_Type_L_I.shape)

    Nuceli_Output_Type_I = Nuceli_Output_Type_L_I
    Nuceli_Label_Type_I = Nuceli_Label_Type_L_I
    Nuceli_Label_Type_I_ = Nuceli_Label_Type_L_I_
    

elif option_Org==4:
    Nuceli_Output_Type_P, Nuceli_Label_Type_P, result_Indices_P = Indices_Type(Organ_Output_P, Organ_Label_P, Type_Value)
    print(Nuceli_Label_Type_P.shape)

    Nuceli_Output_Type = Nuceli_Output_Type_P
    Nuceli_Label_Type = Nuceli_Label_Type_P

    samples_Select = 70
    Nuceli_Output_Type = Nuceli_Output_Type[:, 32:102]

    print('\n\nX_train shape:', Nuceli_Output_Type.shape)
    print('Y_train shape:', Nuceli_Label_Type.shape) 

    Nuceli_Output_Type_P_I, Nuceli_Label_Type_P_I, Nuceli_Label_Type_P_I_ = Organ_Output_P_[result_Indices_P], Organ_Label_P_[result_Indices_P], Organ_Label_P__[result_Indices_P]
    print(Nuceli_Label_Type_P_I.shape)

    Nuceli_Output_Type_I = Nuceli_Output_Type_P_I
    Nuceli_Label_Type_I = Nuceli_Label_Type_P_I
    Nuceli_Label_Type_I_ = Nuceli_Label_Type_P_I_
    
    
elif option_Org==5:
    Nuceli_Output_Type_B, Nuceli_Label_Type_B, result_Indices_B = Indices_Type(Organ_Output_B, Organ_Label_B, Type_Value)
    print(Nuceli_Label_Type_B.shape)

    Nuceli_Output_Type_K, Nuceli_Label_Type_K, result_Indices_K = Indices_Type(Organ_Output_K, Organ_Label_K, Type_Value)
    print(Nuceli_Label_Type_K.shape)

    Nuceli_Output_Type_L, Nuceli_Label_Type_L, result_Indices_L = Indices_Type(Organ_Output_L, Organ_Label_L, Type_Value)
    print(Nuceli_Label_Type_L.shape)

    Nuceli_Output_Type_P, Nuceli_Label_Type_P, result_Indices_P = Indices_Type(Organ_Output_P, Organ_Label_P, Type_Value)
    print(Nuceli_Label_Type_P.shape)

    Nuceli_Output_Type = np.concatenate((Nuceli_Output_Type_B, Nuceli_Output_Type_K, Nuceli_Output_Type_L, Nuceli_Output_Type_P), axis=0)
    Nuceli_Label_Type = np.concatenate((Nuceli_Label_Type_B, Nuceli_Label_Type_K, Nuceli_Label_Type_L, Nuceli_Label_Type_P), axis=0)
    
    samples_Select = 70
    Nuceli_Output_Type = Nuceli_Output_Type[:, 32:102]

    print('\n\nX_train shape:', Nuceli_Output_Type.shape)
    print('Y_train shape:', Nuceli_Label_Type.shape) 



    Nuceli_Output_Type_B_I, Nuceli_Label_Type_B_I, Nuceli_Label_Type_B_I_ = Organ_Output_B_[result_Indices_B], Organ_Label_B_[result_Indices_B], Organ_Label_B__[result_Indices_B]
    print(Nuceli_Label_Type_B_I.shape)

    Nuceli_Output_Type_K_I, Nuceli_Label_Type_K_I, Nuceli_Label_Type_K_I_ = Organ_Output_K_[result_Indices_K], Organ_Label_K_[result_Indices_K], Organ_Label_K__[result_Indices_K]
    print(Nuceli_Label_Type_K_I.shape)

    Nuceli_Output_Type_L_I, Nuceli_Label_Type_L_I, Nuceli_Label_Type_L_I_ = Organ_Output_L_[result_Indices_L], Organ_Label_L_[result_Indices_L], Organ_Label_L__[result_Indices_L]
    print(Nuceli_Label_Type_L_I.shape)

    Nuceli_Output_Type_P_I, Nuceli_Label_Type_P_I, Nuceli_Label_Type_P_I_ = Organ_Output_P_[result_Indices_P], Organ_Label_P_[result_Indices_P], Organ_Label_P__[result_Indices_P]
    print(Nuceli_Label_Type_P_I.shape)

    Nuceli_Output_Type_I = np.concatenate((Nuceli_Output_Type_B_I, Nuceli_Output_Type_K_I, Nuceli_Output_Type_L_I, Nuceli_Output_Type_P_I), axis=0)
    Nuceli_Label_Type_I = np.concatenate((Nuceli_Label_Type_B_I, Nuceli_Label_Type_K_I, Nuceli_Label_Type_L_I, Nuceli_Label_Type_P_I), axis=0)
    Nuceli_Label_Type_I_ = np.concatenate((Nuceli_Label_Type_B_I_, Nuceli_Label_Type_K_I_, Nuceli_Label_Type_L_I_, Nuceli_Label_Type_P_I_), axis=0)



print('\n\nX_train shape:', Nuceli_Output_Type_I.shape)
print('Y_train shape:', Nuceli_Label_Type_I.shape) 
print('Y_train shape:', Nuceli_Label_Type_I.shape) 







Select Class

 1-Epithelial 
 2-Lymphocytes 
 3-Macrophoges 
 4-Neutrophils 
1

 Labels Portion 

{0.0: 575, 1.0: 275, 2.0: 70, 3.0: 55}
Class 0: 575
Class 1: 275
Class 2: 70
Class 3: 55
(575,)


X_train shape: (575, 70)
Y_train shape: (575,)
(575, 128, 128)


X_train shape: (575, 128, 128, 3)
Y_train shape: (575, 128, 128)
Y_train shape: (575, 128, 128)


In [9]:
print('\n\nX_train shape:', Nuceli_Output_Type.shape)
print('Y_train shape:', Nuceli_Label_Type.shape) 
print('Y_train shape:', Nuceli_Label_Type)



X_train shape: (540, 70)
Y_train shape: (540,)
Y_train shape: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [10]:
samples_Select = 70

# 1-Epithelial
if option_Class==1:
    Name_T = "E-GLCM GLDM GLRLM GLSZM"
    
# 2-Lymphocytes 
elif option_Class==2:
    Name_T = "L-GLCM GLDM GLRLM GLSZM"
    
# 3-Macrophoges 
elif option_Class==3:
    Name_T = "M-GLCM GLDM GLRLM GLSZM"

# 4-Neutrophils
elif option_Class==4:
    Name_T = "N-GLCM GLDM GLRLM GLSZM"
    
X_train = Nuceli_Output_Type
Y_train = Nuceli_Label_Type


print('\n\nX_train shape:', Nuceli_Output_Type.shape)
print('Y_train shape:', Nuceli_Label_Type.shape) 

print("\n Labels Portion \n")
unique, counts = np.unique(Nuceli_Label_Type, return_counts=True)
abc= dict(zip(unique, counts))
print(abc)



if option_Org==1:
    data_file = 'January/PICKLESNEW/Classify_D_Breast_' + str(Name_T) + '_' + str(samples_Select) + '.pickle'   
    data_file_ = 'January/PICKLESNEW/Classify_D_Breast_I_' + str(Name_T) + '_' + str(samples_Select) + '.pickle'   

elif option_Org==2:
    data_file = 'January/PICKLESNEW/Classify_D_Kidney_' + str(Name_T) + '_' + str(samples_Select) + '.pickle'   
    data_file_ = 'January/PICKLESNEW/Classify_D_Kidney_I_' + str(Name_T) + '_' + str(samples_Select) + '.pickle'   

elif option_Org==3:
    data_file = 'January/PICKLESNEW/Classify_D_Lung_' + str(Name_T) + '_' + str(samples_Select) + '.pickle'   
    data_file_ = 'January/PICKLESNEW/Classify_D_Lung_I_' + str(Name_T) + '_' + str(samples_Select) + '.pickle'   

elif option_Org==4:
    data_file = 'January/PICKLESNEW/Classify_D_Prostate_' + str(Name_T) + '_' + str(samples_Select) + '.pickle'   
    data_file_ = 'January/PICKLESNEW/Classify_D_Prostate_I_' + str(Name_T) + '_' + str(samples_Select) + '.pickle'   

elif option_Org==5:
    data_file = 'January/PICKLESNEW/Classify_D_All_' + str(Name_T) + '_' + str(samples_Select) + '.pickle'   
    data_file_ = 'January/PICKLESNEW/Classify_D_All_I_' + str(Name_T) + '_' + str(samples_Select) + '.pickle'   
    
    
save_pickle(Nuceli_Output_Type, Nuceli_Label_Type, data_file)


print("\n Labels Portion \n")
unique, counts = np.unique(Nuceli_Label_Type_I_, return_counts=True)
abc= dict(zip(unique, counts))
print(abc)


save_pickle__(Nuceli_Output_Type_I, Nuceli_Label_Type_I, Nuceli_Label_Type_I_, data_file_)






X_train shape: (540, 70)
Y_train shape: (540,)

 Labels Portion 

{0.0: 540}

Trying to save pickle to January/PICKLESNEW/Classify_D_Breast_E-GLCM GLDM GLRLM GLSZM_70.pickle

Pickle Saved Successfully!

 Labels Portion 

{0.0: 540}

Trying to save pickle to January/PICKLESNEW/Classify_D_Breast_I_E-GLCM GLDM GLRLM GLSZM_70.pickle

Pickle Saved Successfully!
